# Гиперпицца

In [1]:
from html.parser import HTMLParser
from numpy import array as Arr
import pandas as pd
import os, io, re
import PySimpleGUI as sg

take_money_list, comp_cr2_dict = [], {}
emp_hr_count2_dict, make_deal_dict = {}, {}
emp_hr_total_dict, emp_cost_max = {}, {}
emp_hr_count3_dict, emp_hr_dict = {}, {}
comp_cr3_dict, comp_cr4_dict = {}, {}
emp_hr_count4_dict, buy_rnd4_list = {}, []

## Preprocess text
class MyHTMLParserHP(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result.append(data)

## Get table and result list
def table_result_score(filepath):
    global result
    table_new, result = [], []

    parser = MyHTMLParserHP()
    with io.open(filepath, encoding = 'utf-8') as file:
        for line in file:
            parser.feed(line)
    
    global player_name
    #player_name = [line for line in result if line.startswith('Игрок')][0][7:]
    score = float([line for line in result if line.startswith('Баланс')][0][8:])
    table = result[result.index('Таблица транзакций') + 3 : result.index('Описание транзакций')]
    for i in range(0, len(table), 2):
        table_new.append((table[i], table[i + 1]))

    del result[:result.index('Описание транзакций') + 1]
    levels = [i for i, x in enumerate(result) if x[-2:] == ': ']
    #result = [result[i].lower() for i in range(len(result))]
    result = [result[i].lower() + result[i + 1].lower() if i in levels else result[i] for i in range(len(result))]
    result = [result[i] for i in range(len(result)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result) if x == 'выполнена' or x == 'отклонена']
    indexs.append(len(result))
    result = [result[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result = [x if 'уровень игры: ' in ''.join(x) else \
            x.append('уровень игры: ' + table[table.index(x[1])+1]) or x for x in result]
    #result = [x[:x.index('отклонена')] if 'отклонена' in x else x for x in result]
    result = [x[:x.index('ожидает одобрения участников')] if 'ожидает одобрения участников' in x else x for x in result]
    return(table_new, result, score)

## Rules Function
def rules(code, filename, marks):
    numbers = Arr([0]*8)
    if code == "1.1":
        if ('зафиксировать инженерам распределение приза от корпорации X', '1') in tables[filename]:
            take_money_list.append(filename)
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.2":
        if ('подарить талеры', '1') in tables[filename] \
                and any([y[y.find('кто')+5:] in ' '.join(take_money_list).lower() \
                for y in sum([x for x in results[filename] if 'подарить талеры' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('кто')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "1.3":
        if ('взять у скаута инструмент в аренду', '1') in tables[filename] \
                and any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('cкаут')]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.4":
        if ('покупка готовой продукции', '2') in tables[filename] or \
                any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('инженер')]):
            numbers += Arr([2,0,0,0,0,0,0,0])
    elif code == "1.5":
        if filename in ' '.join(take_money_list) \
                and all([('подарить талеры', k) not in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "1.6":
        if filename in ' '.join(take_money_list) \
                and any([('подарить талеры', k) in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,0,0,0,1,0,0])
    elif code == "2.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('название')]:
            comp_cr2_dict[comp_name].append(filename) if comp_name in comp_cr2_dict.keys() \
                    else comp_cr2_dict.update({comp_name: [filename]})
    elif code == "2.1":
        if all([(action, '2') in tables[filename] for action in \
                ['создать компанию', 'перевести талеры на счет компании', 'нанять сотрудника']]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "2.2":
        if ('перевести талеры на счет компании', '2') in tables[filename] \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "2.3":
        if any([y for y in sum([x for x in results[filename] if 'тип договора: продажа инструмента' in x \
                and 'уровень игры: 2' in x], [])]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.4":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) < 5:
            numbers += Arr([-1,0,-1,0,0,0,0,0])
    elif code == "2.5":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) >= 5:
            numbers += Arr([-1,0,-1,0,0,1,0,0])
    elif code == "2.6":
        emp_hr_count2_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 2' in x], [])])
        emp_cost_max[filename[7:-5].lower()] = max([int(y[y.find('зарплата')+10:]) \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['инженер: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('зарплата')], default=0)
        if emp_hr_count2_dict[filename] > 3:
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.7":
        if ('покупка готовой продукции', '2') in tables[filename] \
                and len([x for x in results[filename] if 'от компании получен  продукт(услуга): отдел закупок корпорации 1'
                in x or 'от компании получен  продукт(услуга): отдел закупок корпорации 2' in x]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.8":
        if any([filename in x for x in comp_cr2_dict.values()]) \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1 \
                and ('перевести талеры на счет компании', '2') not in tables[filename]:
            numbers += Arr([0,0,-2,0,0,0,1,0])
    elif code == "2.9":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,0,0,1,0,0,0,0])
        make_deal_dict[filename] = list(set([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 2' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')]))
    elif code == "3.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('название')]:
            comp_cr3_dict[comp_name].append(filename) if comp_name in comp_cr3_dict.keys() \
                    else comp_cr3_dict.update({comp_name: [filename]})
    elif code == "3.1":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]):
            numbers += Arr([0,0,0,2,0,0,0,0])
    elif code == "3.2":
        make_deal_x = ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if any([y not in make_deal_dict[filename]+['материалы'] for y in make_deal_x]) or make_deal_x.count('материалы') > 2:
            numbers += Arr([1,0,0,2,3,0,0,0])
    elif code == "3.3":
        if ('нанять сотрудника', '3') in tables[filename]:
            numbers += Arr([0,0,0,0,0,0,0,0])
    elif code == "3.4":
        emp_hr_count3_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 3' in x], [])])
        if emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "3.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "3.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "3.7":
        if any([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and all(['от компании получен  продукт(услуга): компания х1' not in x, \
                         'от компании получен  продукт(услуга): компания х2' not in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,1,1,0,0,0,0,0])
    elif code == "3.8":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
        make_deal_dict[filename] = list(set(make_deal_dict[filename]))
    elif code == "3.9":
        emp_hr_total_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        if sum([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) >= 2:
            numbers += Arr([3,0,0,0,3,0,0,0])
    elif code == "3.10":
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]) > 1:
            numbers += Arr([0,0,1,0,1,0,0,0])
    elif code == "3.11":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and any(['от компании получен  продукт(услуга): компания х1' in x, \
                         'от компании получен  продукт(услуга): компания х2' in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.12":
        emp_hr_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and all(['инженер: '+emp not in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.13":
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and any(['инженер: '+emp in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.14":
        revenue_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'заключить сделки' in x \
            and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x])]
        if emp_hr_count3_dict[filename] < 3 and all([(float(x['цена'])*int(x['кол-во'])) > 10 for x in revenue_x]):
            numbers += Arr([0,0,0,0,0,3,0,0])
    elif code == "4.0":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]): #3.1 copy
            numbers += Arr([0,0,2,2,0,0,0,0])
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]) > 1: #3.10 copy
            numbers += Arr([0,0,1,0,1,0,0,0])
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('название')]:
            comp_cr4_dict[comp_name].append(filename) if comp_name in comp_cr4_dict.keys() \
                    else comp_cr4_dict.update({comp_name: [filename]})
        for comp_name in [y[y.find('получатель компания')+21:] \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' in x and 'от компании получен  продукт(услуга): отдел rnd корпорации х' in x], []) \
                if y.startswith('получатель компания')]:
            buy_rnd4_list.append(comp_name)
    elif code == "4.1":
        if ('закрыть компанию', '4') in tables[filename] and \
                max([len(x) for x in comp_cr4_dict.values() if filename in x], default=0) == 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "4.2":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 4' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "4.3":
        emp_hr_count4_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 4' in x], [])])
        if emp_hr_count4_dict[filename] < min(3, emp_hr_count3_dict[filename]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "4.4":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "4.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "4.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr4_dict.items()) if filename in names]
        if any([comp in buy_rnd4_list and buy_rnd4_list.count(comp) >1 for comp in comp_x]):
            numbers += Arr([1,0,1,0,0,0,0,0])
    elif code == "4.7":
        if any([y[y.find('от компании получен  продукт(услуга)')+38:]  in buy_rnd4_list \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' in x and 'ceo: {}'.format(filename[7:-5]).lower() not in x], []) \
                if y.startswith('от компании получен  продукт(услуга)')]):
            numbers += Arr([1,0,0,0,0,0,0,0])
    elif code == "4.8":
        if ('продать компанию', '4') in tables[filename]:
            numbers += Arr([2,2,1,2,2,0,0,0])
    elif code == "4.9":
        if ('продать компанию', '4') in tables[filename] \
                and ('создать компанию', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
            numbers += Arr([1,0,0,0,1,0,0,0])
    elif code == "4.10":
        if ('продать компанию', '4') in tables[filename] \
                and any([y[y.find('инженер')+9:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инженер')]):
            numbers += Arr([-1,-1,-1,-1,-1,0,0,0])
    elif code == "4.11":
        comp_cr_total_x = [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x], []) if y.startswith('название')]
        if len(comp_cr_total_x) > 1:
            if all([y[y.find(':')+2:] in comp_cr_total_x \
                   for y in sum([x for x in results[filename] if 'заключить сделки' in x], []) \
                   if any([y.startswith('от компании'), y.startswith('получатель')])]):
                numbers += Arr([0,0,0,0,0,0,4,0])
    elif code == "4.12":
        if any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,1,0,1,0,0,0,0])
    elif code == "4.13":
        if any([y[y.find('ceo')+5:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,0,0,1,1,0,0,0])
    elif code == "4.14":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
            
    if filename in marks.keys():
        marks[filename] += numbers
    else:
        marks.update({filename: Arr([0]*8)})
        
    for key, value in marks.items():
        marks[key] = [min(y, 10) for y in value]
    
    return(1 if list(numbers) != [0]*8 else 0)

In [2]:
#os.chdir('p:///data')
#os.getcwd()

def main(path):
    os.chdir(path)
    global tables, results, scores
    tables, results, scores = {}, {}, {}
    all_values, marks = {}, {}
    files = [filename for filename in os.listdir() if filename.startswith('_игрок') and filename.endswith('.html')]
    #for n in [1.1, 2.0, 2.6, 2.9, 3.4, 3.8, 3.9, 3.12, 4.0, 4.2]:
    codes = [f"1.{i}" for i in range(1,7)] + [f"2.{i}" for i in range(10)] + [f"3.{i}" for i in range(15)] \
            + [f"4.{i}" for i in range(15)]
    for filename in files:
        try:
            tables[filename], results[filename], scores[filename] = table_result_score(filename)
        except:
            pass
    for code in codes:
        all_values[code] = []
        for filename in files:
            try:
                all_values[code].append(rules(code, filename, marks))
            except Exception as e:
                #print(e)
                all_values[code].append(0)
                marks[filename] = [0]*8
                pass

    final_table = pd.DataFrame(all_values, index=[filename[7:-5] for filename in files])
    final_table['score'] = [list(x)[::-1][3:]+(list(x)[::-1][:3]) for x in marks.values()]
    
    return(final_table)

# PizzaChart

In [3]:
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
from matplotlib.spines import Spine
from matplotlib.path import Path
from math import pi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import PySimpleGUI as sg

In [4]:
def radar_factory(num_vars, frame='circle'):
    
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    
    class RadarAxes(PolarAxes):
        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [5]:
def web1(filename, col_names, numbers):
    dict1 = {col: 0 for col in col_names}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)

    fig, ax = plt.subplots(figsize=(11, 8), subplot_kw=dict(projection='radar'))
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.95, bottom=0.05, left=0.07, right=0.85)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    for i in range(6):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)

    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig(filename, transparent=True)
    return(filename)

In [6]:
col1 = 'Дисконтирование\nустаревших\nрешений'
col2 = 'Давление на\nнезависимого\nпартнера'
col3 = 'Конструирование\n компаний'
col4 = 'Препарирование\nтехнологий'
col5 = 'Скрининг\nшансов'
col6 = 'Борьба за\nпривилегии'
col7 = 'Фиктивная\nдеятельность'
col8 = 'Отношения\nвместо дела'
col_names = [col1, col2, col3, col4, col5, col6, col7, col8]

# Мегаотчет

In [7]:
from docxtpl import DocxTemplate, InlineImage, RichText
from html.parser import HTMLParser
from bs4 import BeautifulSoup as BS
from docx.shared import Mm, Cm
import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import io
import re
import regex
import docx

pd.options.mode.chained_assignment = None

filepath = "Вывод/ОтчетПКПК_{}.docx".format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M"))

file_path = "_сквозной_список_все_личные_счета.html"

blacklist = ['Глеб Черепанов', 'Черепанов Глеб', 'Андрей Богдашёв', 'Богдашёв Андрей', 'Андрей Богдашев', 'Богдашев Андрей',\
        ' Михаил Ковалёв', 'Ковалёв Михаил', 'Михаил Ковалев', 'Ковалев Михаил', 'Столярова Полина', 'Полина Столярова',\
        'Кристина Клюева', 'Клюева Кристина', 'Светлана Андреева', 'Андреева Светлана', 'Владислав Богдан', 'Богдан Владислав',\
        'Червоная Ольга', 'Александр Кожевников']

In [8]:
def window_string(key, text_size, input_size, tooltip=None):
    return ([sg.Text(key, size=(text_size,1)), sg.InputText(key=key, size=(input_size,1), tooltip=tooltip)])

def make_window():    
    layout1 = [
        [sg.HorizontalSeparator()],
        window_string('Номер договора', 26, 30),
        window_string('Число подписания договора', 26, 30, 'числом'),
        window_string('Месяц подписания договора', 26, 30, 'словом'),
        [sg.HorizontalSeparator()],
        window_string('Должность подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Краткое название заказчика', 26, 30),
        window_string('ФИО подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Договор заключён с ', 26, 30),
        window_string('Полная стоимость услуг', 26, 30, 'цифрами'),
        window_string('Полная стоимость услуг словами', 26, 30),
        [sg.HorizontalSeparator()],
        window_string('Дата проведения', 26, 30),
        window_string('Время начала', 26, 30, 'по местному времени'),
        window_string('Время окончания', 26, 30, 'по местному времени'),
        window_string('Адрес проведения', 26, 30),
        [sg.HorizontalSeparator()],
        [sg.Text('Тип игры', size=(26,1)), sg.Combo(['Отборочная игра Стартап-диплом', 'Коммерческая игра для студентов',
                'Коммерческая игра для взрослых', 'Бизнес-Дебют', 'Другое'],
                default_value='Отборочная игра Стартап-диплом', key='gametype', size=(30,1), readonly=True)],
        window_string('Название папки на диске', 26, 30),
        [sg.HorizontalSeparator()]
        ]
    
    layout2 = [
        [sg.HorizontalSeparator()],
        [sg.Text('Папка с данными игры', size=(20,1)),  sg.FolderBrowse('Путь', size=(4,1), target='folder_path'), sg.InputText(size=(30, 1), key='folder_path')],
        [sg.Text('Файл последнего уровня', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='comp_path'), sg.InputText(size=(30, 1), key='comp_path')],
        [sg.Text('Асессор комментарии', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='table_final'), sg.InputText(size=(30, 1), key='table_final')],
        [sg.Text('Асессор заключение', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='conclusion'), sg.InputText(size=(30, 1), key='conclusion')],
        [sg.ProgressBar(100, orientation='h', size=(40,20), key='progress_bar')],
        [sg.HorizontalSeparator()],
        [sg.Output(size=(50,14))],
        [sg.Submit('Запустить'), sg.Cancel('Выйти')],
        [sg.HorizontalSeparator()],
        ]
    
    tab_group_layout = [[sg.Tab('Строки', layout1), sg.Tab('Файлы', layout2)]]
    layout = [[sg.TabGroup(tab_group_layout, key='tab')]]
    
    return sg.Window('Мегаотчет v0.6.6', layout)

In [9]:
def html_read(result_list):
    player_name = result_list[1][7:] if result_list[1][-1] != ' ' else result_list[1][7:-1]
    table = result_list[result_list.index("Таблица транзакций") + 3 : result_list.index("Описание транзакций")]
    table = {table[i]: table.count(table[i]) for i in range(0, len(table), 2)}
    del result_list[:result_list.index("Описание транзакций") + 1]
    levels = [i for i, x in enumerate(result_list) if x[-2:] == ": "]
    result_list = [result_list[i].lower() + result_list[i + 1].lower() if i in levels else result_list[i] for i in range(len(result_list))]
    result_list = [result_list[i] for i in range(len(result_list)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result_list) if x == "выполнена"]
    indexs.append(len(result_list))
    result_list = [result_list[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result_list = [x if "уровень игры: " in ''.join(x) else x.append("уровень игры: " + str(table[x[1]])) or x for x in result_list]
    return (player_name, table, result_list)

def table_load():
    file_path_table = folder_path + file_path
    table = pd.read_html(file_path_table,encoding = 'utf-8')[0]
    table = table[table['Имя'].apply(lambda x: not x.startswith('Бот') and not x.startswith('Аватар') and not x in blacklist)]
    table['Баланс'] = table['Баланс'].apply(int)
    return(table)
    
def counter(values):
    class MyHTMLParser(HTMLParser):
        def handle_data(self, data):
            if not re.match(r'^\s*$', data):
                result_list.append(data)
        
    comps, top_players_count, trans_dict, trans_dict_deal = len(pd.read_html(comp_path, encoding='utf-8')[-1])-2, 0, {}, {}
    trans_all, trans_fire, trans_switch, trans_contr, trans_create, trans_close, trans_money, trans_deal, players_active = 0, 0, 0, 0, 0, 0, 0, 0, []
    comp_closed, comp_sold, comp_invested = set(), set(), set()
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and 'Бот' not in file_name and 'Аватар' not in file_name:
            result_list = []
            parser = MyHTMLParser()
            with io.open(folder_path+file_name, encoding = 'utf-8') as file:
                for line in file:
                    parser.feed(line)
            player_name, result_table, result_list = html_read(result_list)
            
            levels = set([[y[y.find(':')+2:] for y in x if y.startswith('уровень')][0] for x in result_list if 'заключить сделки' in ''.join(x)])
            if len(levels) >1 and player_name in list(table[table['Роль'] == 'Предприниматель']['Имя']):
                 top_players_count += 1
            
            trans_all = len(result_list)
            if 'автоматическое увольнение после окончания уровня' in result_table:
                trans_fire = result_table['автоматическое увольнение после окончания уровня']
            if 'cмена роли' in result_table:
                trans_switch = result_table['cмена роли']
            trans_contr = len([x for x in result_list if 'создать новый контракт' in ''.join(x)])
            trans_create = len([x for x in result_list if 'создать компанию' in ''.join(x)])
            trans_close = len([x for x in result_list if 'закрыть компанию' in ''.join(x)])
            trans_money = len([x for x in result_list if 'перевести талеры на счет компании' in ''.join(x)])
            trans_deal = int(len([x for x in result_list if 'заключить сделки' in ''.join(x)])/2)
            trans_right = trans_create+trans_close+trans_money+int((trans_all-trans_fire-trans_switch-trans_contr)/2)
            trans_dict.update({player_name: trans_right})
            trans_dict_deal.update({player_name: trans_deal})
            
            levels = set([[y[y.find(':')+2:] for y in x if y.startswith('уровень')][0] for x in result_list])
            if any(num in levels for num in ['4', '5']):
                players_active.append(player_name)
            
            comp_closed.update([[y[y.find(':')+2:] for y in x if y.startswith('название')][0] for x in result_list if 'закрыть компанию' in ''.join(x)])            
            comp_sold.update([[y[y.find(':')+2:] for y in x if y.startswith('компания продавец')][0] for x in result_list if 'продать компанию' in ''.join(x)])
            comp_invested.update([[y[y.find(':')+2:] for y in x if y.startswith('в компании')][0] for x in result_list if 'инвестировать' in ''.join(x)])
            #print(trans_create, trans_close, trans_money, trans_all, trans_fire, trans_switch, trans_contr, trans_dict[player_name])
    top_players_part = round(100*top_players_count/len(table))
    top_players_level = 'высокий' if top_players_part >= 0.12 else 'средний'
    eng_active = len(table[(table['Роль']=='Инженер') & (table['Имя'].isin(players_active))])
    ent_active = len(table[(table['Роль']=='Предприниматель') & (table['Имя'].isin(players_active))])
    
    trans_right = sum(trans_dict.values())
    trans_deal = sum(trans_dict_deal.values())
    
    #print(trans_right, trans_deal)
    for key, value in zip(['top_players_part', 'top_players_level', 'comps', 'eng_active', 'ent_active', 'trans_right', 'trans_deal', 'comp_closed', 'comp_sold', 'comp_invested'],
                          [top_players_part, top_players_level, comps, eng_active, ent_active, trans_right, trans_deal, len(comp_closed), len(comp_sold), len(comp_invested)]):
        values[key] = value
    #print(values)
    return(trans_dict)

def search(names, name):
    try:
        name1, name2 = name.split()
        result = [name for name in names if name1 in name.split(' ') and name2 in name.split(' ')]
        if result:
            return max(result)
        else:
            return(name)
    except:
        return(name)

def counter_new(table):
    doc_conclusion = docx.Document(values['conclusion'])
    fullText = []
    for para in doc_conclusion.paragraphs:
        fullText.append(para.text)
    if 'Пример общего заключения асессора:' in fullText:
        fullText = fullText[:fullText.index('Пример общего заключения асессора:')]
    values['conclusion'] = fullText[1:]
    
    values['last_level'] = values['comp_path'][values['comp_path'].find('.html')-1]
    players_last_level_active = 0
    status_final_dict = {}
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and all([name not in file_name for name in ['Бот', 'Аватар', 'Акционер']]):
            status_final_dict[file_name] = 'Выбыший'
            transactions = pd.read_html(folder_path+file_name,encoding = 'utf-8')[0]
            if int(values['last_level']) in set(transactions['Уровень']):
                status_final_dict[file_name] = 'Финалист'
                players_last_level_active += 1
    values['players_last_level_active'] = players_last_level_active
    table['Файл'] = table['Имя'].apply(lambda x: [file_name for file_name in os.listdir(folder_path) if x in file_name][0])
    table['Статус'] = table['Файл'].apply(lambda x: status_final_dict[x])
    table_final = pd.read_excel(values['table_final'])
    table_1 = pd.merge(table, table_final, left_on="Контакт", right_on="Email", how="right")
    table_2 = pd.merge(table, table_final, left_on="Имя", right_on="ФИО", how="right")
    table_1[table_1['Контакт'].isnull()] = table_2.iloc[table_1[table_1.isnull().any(axis=1)].index]
    table_final = table_1
    table = table[~table['Контакт'].isin(table_final['Email'])]
    table = table[~table['Имя'].isin(table_final['ФИО'])]
    table['Эшелон'] = table['Статус'].apply(lambda x: 3 if x == 'Финалист' else 4)
    table_final = pd.concat([table_final, table[table['Эшелон'] == 3], table[table['Эшелон'] == 4]])
    table_final.reset_index(drop=True, inplace=True)
    table_final['№'] = table_final.index + 1
    table_final.fillna('?', inplace=True)
    table_final['Баланс'] = table_final['Баланс'].apply(lambda x: int(x) if isinstance(x, float) else x)
    values['players_all'] = len(table_final)а
    
    col_labels = ['№', 'Фамилия', 'Имя', 'Отчество', 'Контакт', 'Телефон', 'Роль', 'Баланс', 'Статус']
    col_labels_final = ['№', 'Фамилия', 'Имя', 'Отчество', 'Контакт', 'Телефон', 'Роль', 'Баланс', 'Статус']
    values['key'] = 2
    col_1, col_2 = 'Колонка1', 'Колонка2'
    col_labels_final.append(col_1)
    col_labels_final.append(col_2)
    values['col_labels_final'] = col_labels_final
    col_labels = col_labels + ['Комментарий', 'Рекомендация для участника', 'Эшелон']
    
    doc = DocxTemplate("Ввод_new/тело_отчёта_шаблон {}.docx".format(values['key']))
    for key in list(values.keys()):
        if isinstance(key, str) and isinstance(values[key], str):
            key_new = key.replace(" ", "_")
            values[key] = values[key].replace("Нет", "")
            values[key_new] = values.pop(key)
    values['map_2'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level2.png', Cm(10))
    values['map_3'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level3.png', Cm(10))
    values['map_4'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level4.png', Cm(10))
    values['map_5'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level5.png', Cm(10))
    
    echelon_1, echelon_2, echelon_3, echelon_4 = [], [], [], []
    echelon_dict = {1: echelon_1, 2: echelon_2, 3: echelon_3, 4: echelon_4}
    
    for rating, name_1, name, name_2, email, phone, role, score, status_final, comment, recomm, echelon in table_final[col_labels].values[:]:
        try:
            name = str(name)
        except:
            pass
        for file_name in os.listdir(folder_path):
            if name in file_name and file_name.endswith(".html"):
                status = '-'
                transactions = pd.read_html(folder_path + file_name)[0]
                if 'продать компанию' in transactions['Транзакция'].values:
                    status = 'Продал компанию'
                elif 'инвестировать' in transactions['Транзакция'].values:
                    status = 'Получил инвестиции'
                else:
                    try:
                        max_level = str(max(set(transactions['Уровень'])))
                    except:
                        max_level = '0'
                    if max_level == values['last_level']:
                        status = 'Дошёл до конца игры'
                    else:
                        status = 'Выбыл на {} уровне'.format(max_level)

                if pd.isna(email):
                    email = '-'   
                    
                name = name_1+' '+name+' '+name_2
                try:
                    table_final.at[table_final[table_final['Контакт']==email].index[0], col_1] = name_1
                    table_final.at[table_final[table_final['Контакт']==email].index[0], col_2] = name_2
                except:
                    pass

                doc_disk = DocxTemplate("Ввод_new/Профиль {}.{} эшелон - диск.docx".format(echelon, values['key']))
                with open(folder_path + file_name, 'r', encoding='utf-8') as f:
                    html = f.read()
                soup = BS(html, features='lxml')
                for image_link in soup.find_all('img'):
                    if image_link['src'][:9] == 'generated':
                        player_map = InlineImage(doc, folder_path + image_link['src'][:image_link['src'].find('?')], Cm(10))
                        player_map_disk = InlineImage(doc_disk, folder_path + image_link['src'][:image_link['src'].find('?')], Cm(10))
                web1(folder_path+file_name+'.png', col_names, final_table.loc[file_name[7:-5]]['score'])
                pizza = InlineImage(doc, folder_path+file_name+'.png', Cm(10))
                pizza_disk = InlineImage(doc_disk, folder_path+file_name+'.png', Cm(10))
                item = {'name': name, 'email': email, 'date': values['Дата_проведения'], 'role': role, 'score': score, 'rating': rating, 'map': player_map_disk, 'pizza': pizza_disk, 'status': status,
                        'transactions': len(transactions), 'comment': comment, 'recomm': recomm, 'phone': phone, 'col_1': name_1, 'col_2': name_2, 'key': values['key']}
                item.update(key_actions(file_name))
                doc_disk.render({'item': item})
                doc_disk.save('Вывод/эшелон '+str(echelon)+'/'+name+'.docx')
                item['pizza'] = pizza
                item['map'] = player_map
                echelon_dict[echelon].append(item)
                window['progress_bar'].update(round(100*rating/len(table_final)))
    values['table_final'] = table_final[values['col_labels_final']].values
    values['table_1'] = table_final[table_final['Эшелон'] == 1][values['col_labels_final']].values
    values['table_2'] = table_final[table_final['Эшелон'] == 2][values['col_labels_final']].values
    values['table_3'] = table_final[table_final['Эшелон'] == 3][values['col_labels_final']].values
    values['echelon_1'], values['echelon_2'], values['echelon_3'] = echelon_1, echelon_2, echelon_3
    values['echelon_1_len'], values['echelon_2_len'], values['echelon_3_len'] = len(echelon_1), len(echelon_2), len(echelon_3)
    
    doc.render(values)
    doc.save(filepath)
    
    return(table_final)

def key_actions(filename):
    df_t = pd.DataFrame()
    key_actions = {'k1': '', 'k2': '', 'k3': '', 'k4': '', 'k5': ''}
    if final_table.at[filename[7:-5], '1.3']:
        key_actions['k1'] = 'Проинвестировал в повышение производительности инженерного труда'
    elif final_table.at[filename[7:-5], '1.1']:
        key_actions['k1'] = 'Получил приз по результатам конкурса инженерных решений'
    elif final_table.at[filename[7:-5], '1.4']:
        key_actions['k1'] = 'Успешно идентифицировал возможность заработка в качестве сотрудника корпорации'
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k1'] = 'При первой возможности перешёл в предпринимательскую позицию'
    else:
        key_actions['k1'] = 'Пропуск'

    if final_table.at[filename[7:-5], '2.7']:
        key_actions['k2'] = 'Прошёл квалификацию на роль поставщика крупной корпорации'
    elif tables[filename].count(('заключить сделки', '3')) >2 and not final_table.at[filename[7:-5], '1.4']:
        key_actions['k2'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k2'] = 'При первой возможности перешёл в предпринимательскую позицию'
    elif final_table.at[filename[7:-5], '1.4']:
        key_actions['k2'] = 'Успешно освоил рамки поставленной старшим партнёром позиции'
    else:
        key_actions['k2'] = 'Пропуск'


    if tables[filename].count(('заключить сделки', '3')) >2 and not final_table.at[filename[7:-5], '1.4'] \
            and key_actions['k2'] != 'Смог определить рамки и сконструировать деятельность компании' \
            'в условиях высокой неопределённости и отсутствия внешнего заказчика':
        key_actions['k3'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]):
        key_actions['k3'] = 'Успешно провёл переговоры с инвестором и получил инвестиции'
    elif final_table.at[filename[7:-5], '1.4'] and len([x for x in results[filename] if 'заключить сделки' in x \
            and 'тип договора: продажа инструмента' not in x and 'уровень игры: 3' in x]) >1:
        key_actions['k3'] = 'Выполнил задание старшего партнёра на строительство производства'
    else:
        key_actions['k3'] = 'Пропуск'

    if final_table.at[filename[7:-5], '4.14']:
        key_actions['k4'] = 'Организовал производство крупной партии продукции по заказу старшего партнёра'
    elif final_table.at[filename[7:-5], '4.7']:
        key_actions['k4'] = 'Корректно идентифицировал и занял нишу в системе разделения труда'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]) \
            and key_actions['k3'] != 'Успешно провёл переговоры с инвестором и получил инвестиции':
        key_actions['k4'] = 'Успешно провёл переговоры с инвестором и получил инвестиции'
    elif final_table.at[filename[7:-5], '4.8']:
        key_actions['k4'] = 'Продал построенную компанию'
    else:
        key_actions['k4'] = 'Пропуск'

    if final_table.at[filename[7:-5], '4.8'] and key_actions['k4']!= 'Продал построенную компанию':
        key_actions['k5'] ='Продал построенную компанию'
    elif ('найм в качестве директора компании', '4') in tables[filename] and ('продать компанию', '4') in tables[filename] \
            and ('заключить сделки', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
        key_actions['k5'] ='После продажи компании принял предложение о работе наёмным директором \
                и подтвердил эффективность проданной компании'
    elif final_table.at[filename[7:-5], '4.9']:
        key_actions['k5'] ='После продажи компании открыл новую компанию'
    else:
        key_actions['k5'] ='Пропуск'

    return(key_actions)

# UI

In [10]:
import warnings
warnings.filterwarnings('ignore')

try:
    os.mkdir('Вывод/эшелон 1')
    os.mkdir('Вывод/эшелон 2')
    os.mkdir('Вывод/эшелон 3')
    os.mkdir('Вывод/эшелон 4')
except:
    pass


sg.theme('Reddit')
window = make_window()

class MyHTMLParser(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result_list.append(data)

while True:
    event, values = window.read()
    if event in (None, 'Exit', 'Выйти'):
        break
    if event == 'Запустить':
        folder_path = values['folder_path']+'/'
        comp_path = values['comp_path']
        
        for file_name in os.listdir(folder_path):
            new_file_name = file_name.replace('й', 'й').replace('ё', 'ё')
            os.rename(folder_path + file_name, folder_path + new_file_name)
        
        for file_name in os.listdir(folder_path):
            try:
                if file_name.startswith("_игрок") and file_name.endswith(".html"):
                    transactions = pd.read_html(folder_path+file_name,encoding = 'utf-8')[0]
                    result_list = []
                    parser = MyHTMLParser()
                    with io.open(folder_path+file_name, encoding = 'utf-8') as file:
                        for line in file:
                            parser.feed(line)
                    player_name, result_table, result_list = html_read(result_list)
            except:
                print("Удален пустой файл: "+file_name)
                os.remove(folder_path+file_name)
                
        cwd_old = os.getcwd()
        final_table = main(values['folder_path'])
        final_table.to_excel(f'{cwd_old}\\Гипертаблица.xlsx')
        print(f'Файл создан: {cwd_old}\\Гипертаблица.xlsx')
        os.chdir(cwd_old)
                
        table = table_load()
        trans_dict = counter(values)
        table_final = counter_new(table)
        print('Файл создан: '+filepath)
    
window.close()

In [12]:
table_final

,Фамилия,Имя,Отчество,Контакт,Телефон,Роль,Баланс,Файл,Статус,№,ФИО,Email,Комментарий,Рекомендация для участника,Эшелон,Колонка1,Колонка2
0,?,?,?,?,?,?,?,?,?,1,Барсуков Михаил,Osminogr@yandex.ru,"1) Активный и продуктивный участник, продемонс...",Участник продемонстрировал набор базовых компе...,1,NaN,NaN
1,Фамилия 1,Имя 1,Отчество 1,email1@gmail.com,71111111111.0,Предприниматель,300,_игрок_Фамилия 1 Имя 1 Отчество 1.html,Финалист,2,?,?,?,?,3,Фамилия 1,Отчество 1
2,Фамилия8,Имя8,Отчество8,email8@gmail.com,78888888888.0,Инженер,265,_игрок_Фамилия8 Имя8 Отчество8.html,Финалист,3,?,?,?,?,3,Фамилия8,Отчество8
3,Фамилия10,Имя10,Отчество10,Email10@gmail.com,71010101010.0,Директор X,250,_игрок_Фамилия10 Имя10 Отчество10.html,Финалист,4,?,?,?,?,3,Фамилия10,Отчество10
4,Фамилия6,Имя6,Отчество6,email6@gmail.com,76666666666.0,Директор Фонда Z,170,_игрок_Фамилия6 Имя6 Отчество6.html,Финалист,5,?,?,?,?,3,Фамилия6,Отчество6
5,Фамилия9,Имя9,Отчество9,email9@gmail.com,79999999999.0,Инженер,145,_игрок_Фамилия9 Имя9 Отчество9.html,Финалист,6,?,?,?,?,3,Фамилия9,Отчество9
6,Фамилия5,Имя5,Отчество5,email5@gmail.com,75555555555.0,Инвестиционный менеджер,134,_игрок_Фамилия5 Имя5 Отчество5.html,Финалист,7,?,?,?,?,3,Фамилия5,Отчество5
7,Фамилия7,Имя7,Отчество7,email7@gmail.com,77777777777.0,Предприниматель,80,_игрок_Фамилия7 Имя7 Отчество7.html,Финалист,8,?,?,?,?,3,Фамилия7,Отчество7
8,Фамилия4,Имя4,Отчество4,email4@gmail.com,74444444444.0,Инженер,42,_игрок_Фамилия4 Имя4 Отчество4.html,Финалист,9,?,?,?,?,3,Фамилия4,Отчество4
9,Фамилия3,Имя3,Отчество3,email3@gmail.com,73333333333.0,Предприниматель,-47,_игрок_Фамилия3 Имя3 Отчество3.html,Финалист,10,?,?,?,?,3,Фамилия3,Отчество3


pyinstaller -F -w -i "icon3.ico" Мегаотчетv0.6.6.py --exclude-module PyQt5